In [1]:
import pandas as pd
!pip install Sastrawi
import numpy as np
import re
import string
import nltk
nltk.download('punkt')
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.2 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = "/content/drive/MyDrive/Skripsi/Data/datafinallast.xlsx"
df = pd.read_excel(data)
df

,full_text,sentimen,cleansed,tokenized,stemmed,stopwords,fix
0,Resmi Naik BPJS Kesehatan Ungkap Alasan Penyes...,1,resmi naik bpjs kesehatan ungkap alasan penyes...,"['resmi', 'naik', 'bpjs', 'kesehatan', 'ungkap...","['resmi', 'naik', 'bpjs', 'sehat', 'ungkap', '...","['resmi', 'naik', 'bpjs', 'sehat', 'alas', 's...",resmi naik bpjs sehat alas sesuai iur kelas
1,Ketika belanja di warung tetangga seribu perak...,-1,ketika belanja warung tetangga seribu perak ju...,"['ketika', 'belanja', 'warung', 'tetangga', 's...","['ketika', 'belanja', 'warung', 'tetangga', 'r...","['belanja', 'warung', 'tetangga', 'ribu', 'per...",belanja warung tetangga ribu perak tawar gigih...
2,@PutraWadapi Akan banyak Potongan salah satuny...,1,akan banyak potongan salah satunya potongan b...,"['akan', 'banyak', 'potongan', 'salah', 'satun...","['akan', 'banyak', 'potong', 'salah', 'satu', ...","['potong', 'salah', 'potong', 'bpjs', 'naik',...",potong salah potong bpjs naik usutpredatorkm u...
3,Berita soal iuran BPJS Kesehatan naik per 1 Ja...,1,berita soal iuran bpjs kesehatan naik per janu...,"['berita', 'soal', 'iuran', 'bpjs', 'kesehatan...","['berita', 'soal', 'iur', 'bpjs', 'sehat', 'na...","['berita', 'iur', 'bpjs', 'sehat', 'naik', 'j...",berita iur bpjs sehat naik januari rajin tempe...
4,pagi-pagi dapet email #BPJSNaik lagi ðŸ˜Œ http...,1,pagipagi dapet email bpjsnaik lagi ˜,"['pagipagi', 'dapet', 'email', 'bpjsnaik', 'la...","['pagipagi', 'dapet', 'email', 'bpjsnaik', 'la...","['pagipagi', 'dapet', 'email', 'bpjsnaik', '']",pagipagi dapet email bpjsnaik
...,...,...,...,...,...,...,...
1865,@4nneve @jokowi Rakyat sdh dijajah dgn Kebijak...,-1,rakyat sdh dijajah dgn kebijakan pro kapitali...,"['rakyat', 'sdh', 'dijajah', 'dgn', 'kebijakan...","['rakyat', 'sdh', 'jajah', 'dgn', 'bijak', 'pr...","['rakyat', 'sdh', 'jajah', 'dgn', 'bijak', 'pr...",rakyat sdh jajah dgn bijak pro kapitalis impor...
1866,@aldrin_neil @sudjiwotedjo Dipreteli sebelah m...,-1,dipreteli sebelah manaputusan pengadilan tent...,"['dipreteli', 'sebelah', 'manaputusan', 'penga...","['pretel', 'belah', 'manaputusan', 'adil', 'te...","['pretel', 'belah', 'manaputusan', 'adil', 'bp...",pretel belah manaputusan adil naik bpjs aja ak...
1867,@febridiansyah @OmbudsmanRI137 Disini problem ...,-1,disini problem besarnya gak yakin rekomendasi...,"['disini', 'problem', 'besarnya', 'gak', 'yaki...","['sini', 'problem', 'besar', 'gak', 'yakin', '...","['problem', 'gak', 'rekomendasi', 'laksana', '...",problem gak rekomendasi laksana presiden laksa...
1868,@mrshananto BPJS kesehatan THR 1 bulan gaji ta...,1,bpjs kesehatan thr bulan gaji tabungan emas d...,"['bpjs', 'kesehatan', 'thr', 'bulan', 'gaji', ...","['bpjs', 'sehat', 'thr', 'bulan', 'gaji', 'tab...","['bpjs', 'sehat', 'thr', 'gaji', 'tabung', 'em...",bpjs sehat thr gaji tabung emas digital naik gaji


In [4]:
df.drop(['full_text','cleansed','tokenized','stemmed','stopwords'], axis=1, inplace=True)
df

,sentimen,fix
0,1,resmi naik bpjs sehat alas sesuai iur kelas
1,-1,belanja warung tetangga ribu perak tawar gigih...
2,1,potong salah potong bpjs naik usutpredatorkm u...
3,1,berita iur bpjs sehat naik januari rajin tempe...
4,1,pagipagi dapet email bpjsnaik
...,...,...
1865,-1,rakyat sdh jajah dgn bijak pro kapitalis impor...
1866,-1,pretel belah manaputusan adil naik bpjs aja ak...
1867,-1,problem gak rekomendasi laksana presiden laksa...
1868,1,bpjs sehat thr gaji tabung emas digital naik gaji


In [5]:
# menentukan kolom x dan y
X = df['fix']
y = df['sentimen'].astype(int)

In [6]:
# TRAIN TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y, random_state=0)

In [7]:
X_train.shape

(1496,)

In [8]:
X_test.shape

(374,)

In [9]:
#TF-IDF
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.fit_transform(X_test)

In [10]:
import sklearn
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC


In [13]:
from sklearn import svm
from sklearn.model_selection import cross_val_predict

In [14]:
linear = svm.SVC(kernel="linear")
y_pred = cross_val_predict(linear,X_test, y_test, cv=10)

In [15]:
#accuracy score
print("accuracy = ")
print(accuracy_score(y_test,y_pred))

#precision score
print("precision = ")
print(precision_score(y_test,y_pred, pos_label=1))

#recall score
print("recall = ")
print(recall_score(y_test,y_pred, pos_label=1))

#f1 score
print("f1 score = ")
print(f1_score(y_test,y_pred, pos_label=1))

#confusion matrix
print(confusion_matrix(y_test, y_pred))

accuracy = 
0.7299465240641712
precision = 
0.7597402597402597
recall = 
0.6464088397790055
f1 score = 
0.698507462686567
[[156  37]
 [ 64 117]]
